In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle

import time

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

from tensorflow.keras.applications import Xception, MobileNetV2
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# GPU config if using gpu then uncomment following lines
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
# Setup the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=16000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [16]:
fps = '1FPS'
weight_dir = '../../data/models/multi_task'
numpy_dir = '../../data/DAiSEE/' + fps + '/dataImages/'

In [5]:
def show_batch(image, label):
    image = image.numpy()
    plt.figure(figsize=(15, 15))
    for i in range(batch_size):
        plt.subplot(6, 6, i + 1)
        imgtitle = [label["y1"][i].numpy().item(),
                    label["y2"][i].numpy().item(),
                    label["y3"][i].numpy().item(),
                    label["y4"][i].numpy().item()]
        plt.imshow(np.uint8(image[i] * 255))
        plt.title(imgtitle, fontsize=8)
        plt.axis('off')
    plt.show()

In [6]:
def parse_function(filepath, label):
    image = tf.io.read_file(filepath)
    image = tf.io.decode_jpeg(contents=image, channels=3)
    image = tf.image.convert_image_dtype(image=image, dtype=tf.float32)
    image = tf.image.resize(images=image,
                            size=[img_width, img_height],
                            method=tf.image.ResizeMethod.BILINEAR)
    return image, label

In [7]:
def get_dataset(usage, numpy_dir):
    numpy_dir = Path(numpy_dir)
    x = np.load(numpy_dir / f'x_{usage.lower()}.npy', allow_pickle=True)
    y = np.load(numpy_dir / f'y_{usage.lower()}.npy')
    
    x, y = shuffle(x, y, random_state=0)
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (x, {"y1": y[:, :1], "y2": y[:, 1:2],
             "y3": y[:, 2:3], "y4": y[:, :3:4]})
    )
    dataset = dataset.map(map_func=parse_function, num_parallel_calls=autotune)
    if usage == 'Train':
        dataset = dataset.shuffle(buffer_size=shuffle_buffer_size,
                                  reshuffle_each_iteration=True)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(autotune)
    return dataset

In [8]:
autotune = tf.data.experimental.AUTOTUNE
img_width = 224
img_height = 224
batch_size = 16
shuffle_buffer_size = 2000

finetune_at = 116
base_learning_rate = 0.0001
old_epoch = 0
epochs = 10

In [9]:
def create_model(pre_trained, image_size, weights, epochs):

    # create the base pre-trained model, don't include classification layer
    base_model = pre_trained(input_shape = (image_size, image_size, 3),
                             include_top = False, 
                             weights = weights)

    # Freeze all layers (up to layer 100)
    #for layer in base_model.layers:
    for layer in base_model.layers[:126]: 
        layer.trainable = False

    # Add some layers    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation="relu", name="fc1")(x)
    x = Dense(64, activation="relu", name="fc2")(x)
    boredom = Dense(4, activation='softmax', name="y1")(x)
    engagement = Dense(4, activation='softmax', name="y2")(x)
    confusion = Dense(4, activation='softmax', name="y3")(x)
    frustration = Dense(4, activation='softmax', name="y4")(x)
    model = Model(inputs=base_model.input,
                  outputs=[boredom, engagement, confusion, frustration])

    return model

In [10]:
def run_model(model_name, model, lr, decay):
    
    # checkpoint callback
    timestr = time.strftime("%Y%m%d-%H%M%S")
    dir_name = weight_dir + '/'
    best_model_file = dir_name + model_name + '_' + timestr + '_{epoch}.hdf5'

    checkpoint = ModelCheckpoint(best_model_file,
                                 monitor='accuracy',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='max')

    # early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')   

    callbacks = [checkpoint, early_stopping]

    # compilation
    model.compile(optimizer=Adam(lr=lr, decay=decay),
                  loss={"y1": SparseCategoricalCrossentropy(from_logits=True),
                        "y2": SparseCategoricalCrossentropy(from_logits=True),
                        "y3": SparseCategoricalCrossentropy(from_logits=True),
                        "y4": SparseCategoricalCrossentropy(from_logits=True)},
                  metrics={"y1": "sparse_categorical_accuracy",
                           "y2": "sparse_categorical_accuracy",
                           "y3": "sparse_categorical_accuracy",
                           "y4": "sparse_categorical_accuracy"})

    # train
    history = model.fit(
                train_ds,
                epochs=epochs,
                callbacks = callbacks,
                validation_data=validation_ds)
    
    return history

In [11]:
pre_trained_model, image_size, model_name = MobileNetV2, 224, 'MobileNetV2'
weights ='imagenet'
epochs = 100
batch_size = 32
lr = 0.0001
decay = 1e-6

# call the functions
train_ds = get_dataset("Train", numpy_dir)
validation_ds = get_dataset("Validation", numpy_dir)


In [12]:
model = create_model(pre_trained_model, image_size, weights, epochs)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [13]:
history = run_model(model_name, model, lr, decay)

Epoch 1/100
1675/1675 [==============================] - 66s 39ms/step - loss: 4.4909 - y1_loss: 1.1704 - y2_loss: 1.1002 - y3_loss: 1.0524 - y4_loss: 1.1679 - y1_sparse_categorical_accuracy: 0.5619 - y2_sparse_categorical_accuracy: 0.6359 - y3_sparse_categorical_accuracy: 0.6900 - y4_sparse_categorical_accuracy: 0.5633 - val_loss: 5.1211 - val_y1_loss: 1.3668 - val_y2_loss: 1.2449 - val_y3_loss: 1.1268 - val_y4_loss: 1.3826 - val_y1_sparse_categorical_accuracy: 0.3407 - val_y2_sparse_categorical_accuracy: 0.4733 - val_y3_sparse_categorical_accuracy: 0.6100 - val_y4_sparse_categorical_accuracy: 0.3179
Epoch 2/100
1675/1675 [==============================] - 64s 38ms/step - loss: 4.3435 - y1_loss: 1.1214 - y2_loss: 1.0671 - y3_loss: 1.0338 - y4_loss: 1.1213 - y1_sparse_categorical_accuracy: 0.6155 - y2_sparse_categorical_accuracy: 0.6714 - y3_sparse_categorical_accuracy: 0.7079 - y4_sparse_categorical_accuracy: 0.6155 - val_loss: 5.1231 - val_y1_loss: 1.3552 - val_y2_loss: 1.1715 - val_

In [14]:
print(weight_dir)

../../models/multi_task


In [17]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = weight_dir + '/MobileNetv2_' + timestr + '_10.hdf5'
print(save_name)

model.save(save_name)

../../data/models/multi_task/MobileNetv2_20201205-190003_10.hdf5


In [18]:
# Evaluate results on test set

class_names = ['Boredom', 'Engagement', 'Confusion', 'Frustration']

test_ds = get_dataset("Test", numpy_dir)
results = model.evaluate(test_ds)


558/558 [==============================] - 15s 27ms/step - loss: 5.0181 - y1_loss: 1.3060 - y2_loss: 1.2433 - y3_loss: 1.1625 - y4_loss: 1.3063 - y1_sparse_categorical_accuracy: 0.4088 - y2_sparse_categorical_accuracy: 0.4740 - y3_sparse_categorical_accuracy: 0.5748 - y4_sparse_categorical_accuracy: 0.4042


In [19]:
for i, acc in enumerate(results[5:]):
    print(f"{class_names[i]}: {acc*100}")

Boredom: 40.88004529476166
Engagement: 47.404709458351135
Confusion: 57.4775755405426
Frustration: 40.42040407657623
